In [1]:
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context='talk', style='ticks', color_codes=True)

%matplotlib inline

In [3]:
df = pd.read_csv('data/qm9.csv.gz')
df.index = df['index'].apply(lambda x: 'gdb_{}'.format(x))

In [4]:
import warnings
from tqdm import tqdm

import gzip
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import ForwardSDMolSupplier

In [5]:
f = gzip.open('data/gdb9.sdf.gz')

mol_supplier = ForwardSDMolSupplier(f, removeHs=False)
from itertools import islice

mols = []
total_mols = 5000

for mol in tqdm(islice(mol_supplier, 0, total_mols), total=total_mols):
    if mol:
        mols += [(mol.GetProp('_Name'), mol, mol.GetNumAtoms())]
        
mols = pd.DataFrame(mols, columns=['mol_id', 'Mol', 'n_atoms'])

100%|██████████| 5000/5000 [00:01<00:00, 4812.64it/s]


In [6]:
valid = mols.sample(frac=.1, random_state=0)
train = mols[~mols.mol_id.isin(valid.mol_id)].sample(frac=1., random_state=0)

mols = mols.set_index('mol_id')

In [7]:
from nfp.preprocessing import MolPreprocessor, GraphSequence
from sklearn.preprocessing import RobustScaler

Using TensorFlow backend.


In [9]:
def rbf_expansion(distances, mu=0, delta=0.1, kmax=150):
    k = np.arange(0, kmax)
    logits = -(np.atleast_2d(distances).T - (-mu + delta * k))**2 / delta
    return np.exp(logits)

def atomic_number_tokenizer(atom):
    return atom.GetAtomicNum()

In [10]:
# Preprocess molecules
preprocessor = MolPreprocessor(
    n_neighbors=48, atom_features=atomic_number_tokenizer)
inputs_train = preprocessor.fit(train.Mol)
inputs_valid = preprocessor.predict(valid.Mol)

100%|██████████| 496/496 [00:00<00:00, 1310.12it/s]


In [11]:
def precalc_rbfs(inputs):
    for item in tqdm(inputs):
        item['distance_rbf'] = rbf_expansion(item['distance'])
        del item['distance']
    return inputs

inputs_train = precalc_rbfs(inputs_train)
inputs_valid = precalc_rbfs(inputs_valid)

100%|██████████| 496/496 [00:00<00:00, 677.49it/s]


In [12]:
# Preprocess y values
y_train = df.reindex(train.mol_id).U0.values.reshape(-1, 1)
y_valid = df.reindex(valid.mol_id).U0.values.reshape(-1, 1)

Train a quick group-contribution model to get initial values for enthalpies per atom

In [13]:
from collections import Counter
from sklearn.linear_model import LinearRegression

X = pd.DataFrame([Counter(row['atom']) for row in inputs_train]).fillna(0)

model = LinearRegression(fit_intercept=False)
model.fit(X, y_train)

atom_contributions = pd.Series(model.coef_.flatten(), index=X.columns)
atom_contributions = atom_contributions.reindex(np.arange(preprocessor.atom_classes)).fillna(0)

In [14]:
# y_scaler = RobustScaler()
# y_train_scaled = y_scaler.fit_transform(y_train)
# y_valid_scaled = y_scaler.transform(y_valid)

# Create batch iterators
batch_size = 32
train_generator = GraphSequence(inputs_train, y_train, 32)
valid_generator = GraphSequence(inputs_valid, y_valid, 32)

In [15]:
# Define Keras model
import keras
import keras.backend as K

from keras.layers import (
    Input, Embedding, Dense, BatchNormalization, Concatenate, Multiply, Add)

from keras.models import Model

from nfp.layers import (MessageLayer, GRUStep, Squeeze, EdgeNetwork,
                        ReduceAtomToMol, ReduceBondToAtom, GatherAtomToBond)
from nfp.models import GraphModel

In [16]:
# Raw (integer) graph inputs
node_graph_indices = Input(shape=(1,), name='node_graph_indices', dtype='int32')
atom_types = Input(shape=(1,), name='atom', dtype='int32')
distance_rbf = Input(shape=(150,), name='distance_rbf', dtype='float32')
connectivity = Input(shape=(2,), name='connectivity', dtype='int32')

squeeze = Squeeze()

snode_graph_indices = squeeze(node_graph_indices)
satom_types = squeeze(atom_types)

# Initialize RNN and MessageLayer instances
atom_features = 16

# Initialize the atom states
atom_state = Embedding(
    preprocessor.atom_classes,
    atom_features, name='atom_embedding')(satom_types)

atomwise_energy = Embedding(
    preprocessor.atom_classes, 1, name='atomwise_energy',
    embeddings_initializer=keras.initializers.constant(atom_contributions.values)
)(satom_types)


bond_state = distance_rbf

def message_block(atom_state, bond_state, connectivity):

    source_atom_gather = GatherAtomToBond(1)
    target_atom_gather = GatherAtomToBond(0)

    source_atom = source_atom_gather([atom_state, connectivity])
    target_atom = target_atom_gather([atom_state, connectivity])

    # Edge update network
    bond_state = Concatenate()([source_atom, target_atom, bond_state])
    bond_state = Dense(2*atom_features, activation='softplus')(bond_state)
    bond_state = Dense(atom_features)(bond_state)

    # message function
    bond_state = Dense(atom_features)(bond_state)
    bond_state = Dense(atom_features)(bond_state)
    source_atom = Dense(atom_features)(source_atom)    
    messages = Multiply()([source_atom, bond_state])
    messages = ReduceBondToAtom(reducer='sum')([messages, connectivity])
    
    # state transition function
    messages = Dense(atom_features, activation='softplus')(messages)
    messages = Dense(atom_features)(messages)
    atom_state = Add()([atom_state, messages])
    
    return atom_state, bond_state

for _ in range(3):
    atom_state, bond_state = message_block(atom_state, bond_state, connectivity)
    
atom_state = Dense(atom_features//2, activation='softplus')(atom_state)
atom_state = Dense(1)(atom_state)
atom_state = Add()([atom_state, atomwise_energy])
output = ReduceAtomToMol(reducer='sum')([atom_state, snode_graph_indices])

model = GraphModel([
    node_graph_indices, atom_types, distance_rbf, connectivity], [output])

model.compile(optimizer=keras.optimizers.Adam(lr=5E-4), loss='mae')
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
atom (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
squeeze_1 (Squeeze)             (None,)              0           node_graph_indices[0][0]         
                                                                 atom[0][0]                       
__________________________________________________________________________________________________
atom_embedding (Embedding)      (None, 16)           112         squeeze_1[1][0]                  
__________________________________________________________________________________________________
connectivity (InputLayer)       (None, 2)            0                                            
__________

In [17]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    hist = model.fit_generator(
        train_generator, validation_data=valid_generator, epochs=10)

Epoch 1/10
140/140 [==============================] - 6s 46ms/step - loss: 24.3328 - val_loss: 2.3053
Epoch 2/10
140/140 [==============================] - 5s 38ms/step - loss: 2.9814 - val_loss: 2.6666
Epoch 3/10
140/140 [==============================] - 6s 39ms/step - loss: 1.9592 - val_loss: 1.6980
Epoch 4/10
140/140 [==============================] - 6s 41ms/step - loss: 1.3003 - val_loss: 1.1896
Epoch 5/10
140/140 [==============================] - 5s 38ms/step - loss: 1.0549 - val_loss: 0.9993
Epoch 6/10
140/140 [==============================] - 6s 40ms/step - loss: 1.1060 - val_loss: 1.4312
Epoch 7/10
140/140 [==============================] - 6s 40ms/step - loss: 1.1539 - val_loss: 1.4891
Epoch 8/10
140/140 [==============================] - 7s 46ms/step - loss: 1.0316 - val_loss: 0.8721
Epoch 9/10
140/140 [==============================] - 6s 46ms/step - loss: 0.9200 - val_loss: 0.9274
Epoch 10/10
140/140 [==============================] - 6s 46ms/step - loss: 0.8966 - val_l

In [ ]:
#plt.semilogy(hist.history['val_loss'])
plt.semilogy(hist.history['loss'])
plt.semilogy(hist.history['val_loss'])

In [ ]:
y_pred_train = model.predict_generator(RBFSequence(inputs_train, batch_size=32, shuffle=False))
y_pred_valid = model.predict_generator(RBFSequence(inputs_valid, batch_size=32, shuffle=False))

In [ ]:
plt.plot(y_valid, y_pred_valid, '.', ms=2)

In [ ]:
plt.plot(y_train, y_pred_train, '.', ms=2)

In [ ]:
model.save('temp/test3d.h5')

from keras.models import load_model
from nfp import custom_layers

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    model = load_model('temp/test3d.h5', custom_objects=custom_layers)

model.evaluate_generator(valid_generator)